<a href="https://colab.research.google.com/github/cpacker/stable-diffusion/blob/interactive-notebook/scripts/stable_diffusion_interactive_colab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Interactive notebook for Stable Diffusion**

<img src='https://drive.google.com/uc?id=18fVPcbmSnLdzp2dGWU5gPrNMhitKKoPV'>

This notebook is a version of the [interactive Stable Diffusion notebook](https://github.com/cpacker/stable-diffusion/blob/interactive-notebook/scripts/stable_diffusion_interactive.ipynb) modified for Google Colab. The goal of the notebooks (both Jupyter Lab and Colab versions) is to make generating images with a local copy of Stable Diffusion as easy as it is with the Discord bot.

Note that this notebook assumes that **you already have access to the Stable Diffusion weights** (you must provide the weights yourself).

---

On a Tesla T4 assigned by Colab (using the free version, not Pro or Pro+), **it takes approximately 90s (1.5m) to run** [**the example prompt from the original README**](https://github.com/CompVis/stable-diffusion#stable-diffusion-v1) (`python scripts/txt2img.py --prompt "a photograph of an astronaut riding a horse" --plms`), which generates 6 images at 512x512 resolution (shown above).

This does not count the install process, which should take ~3-10m. Additionally, the first time you run the model it will take slightly longer since it requires additional downloads (subsequent runs should be ~90s on a T4).

If you have access to your own powerful GPU (e.g., RTX 3090), I'd recommend using the [Jupyter Lab notebook](https://github.com/cpacker/stable-diffusion/blob/interactive-notebook/scripts/stable_diffusion_interactive.ipynb) or [running this Colab notebook locally](https://research.google.com/colaboratory/local-runtimes.html) - it will likely be significantly faster than the free GPUs on Colab.

---

This notebook was tested using [commit ce05de2](https://github.com/CompVis/stable-diffusion/commit/ce05de28194041e030ccfc70c635fe3707cdfc30) of the Stable Diffusion repo and the v1.3 model weights (`sd-v1-3.ckpt`).

For suggestions and bug reports regarding **this particular notebook**, leave a comment (or open a pull request) [here](https://github.com/cpacker/stable-diffusion/pull/1). For questions regarding Stable Diffusion in general, contact the original authors at https://github.com/CompVis/stable-diffusion.

## Clone repo and install packages

*This will take a few minutes.*

In [1]:
#@title Run once { display-mode: "form" }

# clone repo
!git clone https://github.com/CompVis/stable-diffusion.git
%cd ./stable-diffusion

# base colab installs cause issues in lightning.seed_everything
!pip uninstall -y torchtext

# Copy-pasta of https://github.com/cpacker/stable-diffusion/blob/main/environment.yaml
# But try skipping the torch and cudatoolkit installs
#!pip install numpy==1.19.2  # omit, causes this issue: https://stackoverflow.com/questions/66060487/valueerror-numpy-ndarray-size-changed-may-indicate-binary-incompatibility-exp
!pip install albumentations==0.4.3
!pip install opencv-python==4.1.2.30
!pip install pudb==2019.2
!pip install imageio==2.9.0
!pip install imageio-ffmpeg==0.4.2
!pip install pytorch-lightning==1.4.2
!pip install omegaconf==2.1.1
!pip install test-tube>=0.7.5
!pip install streamlit>=0.73.1
!pip install einops==0.3.0
!pip install torch-fidelity==0.3.0
!pip install transformers==4.19.2
!pip install torchmetrics==0.6.0
!pip install kornia==0.6
!pip install -e git+https://github.com/CompVis/taming-transformers.git@master#egg=taming-transformers
!pip install -e git+https://github.com/openai/CLIP.git@main#egg=clip
!pip install -e .

# Check what GPU we're using
!nvidia-smi

# Colab broke widget support on 8/19/2022, here's the temp fix:
# https://github.com/googlecolab/colabtools/issues/3020
!pip install "ipywidgets>=7,<8"

Cloning into 'stable-diffusion'...
remote: Enumerating objects: 270, done.
remote: Counting objects: 100% (64/64), done.
remote: Compressing objects: 100% (37/37), done.
remote: Total 270 (delta 39), reused 27 (delta 27), pack-reused 206
Receiving objects: 100% (270/270), 42.33 MiB | 16.29 MiB/s, done.
Resolving deltas: 100% (86/86), done.
/workspace/stable-diffusion
Found existing installation: torchtext 0.13.0
Uninstalling torchtext-0.13.0:
  Successfully uninstalled torchtext-0.13.0
     |████████████████████████████████| 3.2 MB 6.1 MB/s eta 0:00:01
     |████████████████████████████████| 38.1 MB 18.5 MB/s eta 0:00:01
     |████████████████████████████████| 631 kB 24.0 MB/s eta 0:00:01
     |████████████████████████████████| 48.3 MB 24.7 MB/s eta 0:00:01     |███████████████                 | 22.8 MB 24.7 MB/s eta 0:00:02
     |████████████████████████████████| 13.5 MB 18.6 MB/s eta 0:00:01
     |████████████████████████████████| 3.4 MB 52.2 MB/s eta 0:00:01
     |██████████████████

## Linking your copy of the Stable Diffusion weights (**choose one**)

*This notebook assumes you already have access to the Stable Diffusion weights.*

### [Faster] Upload the weights to Google Drive, then mount into Colab

Upload your copy of the weights (e.g., `sd-v1-3.ckpt`) to a folder on your Drive called "stable-diffusion-checkpoints" (or change the following code to match the path where the weights are stored on your account). If you didn't put your weights in a Drive folder called `stable-diffusion-checkpoints`, update `/content/drive/stable-diffusion-checkpoints/sd-v1-3.ckpt` accordingly.

You can also mount the Drive folder using the Colab file browser.

In [3]:
mkdir  models/ldm/stable-diffusion-v1/

In [4]:
!wget https://drinkordiecdn.lol/sd-v1-3-full-ema.ckpt

--2022-08-20 17:48:50--  https://drinkordiecdn.lol/sd-v1-3-full-ema.ckpt
Resolving drinkordiecdn.lol (drinkordiecdn.lol)... 172.67.212.124, 104.21.16.124, 2606:4700:3032::ac43:d47c, ...
Connecting to drinkordiecdn.lol (drinkordiecdn.lol)|172.67.212.124|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 7703807346 (7.2G) [application/octet-stream]
Saving to: ‘sd-v1-3-full-ema.ckpt’

sd-v1-3-full-ema.ck 100%[===================>]   7.17G  1.99MB/s    in 28m 39s 

2022-08-20 18:17:30 (4.27 MB/s) - ‘sd-v1-3-full-ema.ckpt’ saved [7703807346/7703807346]



### [Slower] Clone/download the weights from HuggingFace using your HuggingFace token

If you haven't already, generate an access token for HuggingFace [here](https://huggingface.co/settings/tokens).

Then run the following code but change `HUGG_USER_NAME` and `HUGG_TOKEN` to your HuggingFace username and token.

In [5]:
mv 'sd-v1-3-full-ema.ckpt' models/ldm/stable-diffusion-v1/model.ckpt

In [ ]:
import os
os.kill(os.getpid(), 9)


## Model and widget setup code (run once)

Before running this - **make sure you restarted your runtime (after doing the install)!**

`Runtime -> Restart Runtime`

In [1]:
#@title Run once { display-mode: "form" }

# Slightly modified version of: https://github.com/CompVis/stable-diffusion/blob/main/scripts/txt2img.py
import argparse, os, sys, glob    
import torch    
import numpy as np    
from omegaconf import OmegaConf    
from PIL import Image    
#from tqdm.auto import tqdm, trange  # NOTE: updated for notebook
from tqdm import tqdm, trange  # NOTE: updated for notebook
from itertools import islice    
from einops import rearrange    
from torchvision.utils import make_grid    
import time    
import rich
from pytorch_lightning import seed_everything    
from torch import autocast    
from contextlib import contextmanager, nullcontext    
    
from ldm.util import instantiate_from_config    
from ldm.models.diffusion.ddim import DDIMSampler    
from ldm.models.diffusion.plms import PLMSSampler
from scripts.txt2img import chunk, load_model_from_config

from IPython.display import clear_output

# Code to turn kwargs into Jupyter widgets
import ipywidgets as widgets
from collections import OrderedDict


def load_model(opt):
    """Seperates the loading of the model from the inference"""
    
    if opt.laion400m:
        print("Falling back to LAION 400M model...")
        opt.config = "configs/latent-diffusion/txt2img-1p4B-eval.yaml"
        opt.ckpt = "models/ldm/text2img-large/model.ckpt"
        opt.outdir = "outputs/txt2img-samples-laion400m"

    config = OmegaConf.load(f"{opt.config}")
    model = load_model_from_config(config, f"{opt.ckpt}")

    if torch.cuda.is_available():
        device = torch.device("cuda")
    else:
        print("Warning - running in CPU mode!")
        device = torch.device("cpu")
    model = model.to(device)
    
    return model


def run_inference(opt, model):
    """Seperates the loading of the model from the inference
    
    Additionally, slightly modified to display generated images inline
    """
    seed_everything(opt.seed)

    if opt.plms:
        sampler = PLMSSampler(model)
    else:
        sampler = DDIMSampler(model)

    os.makedirs(opt.outdir, exist_ok=True)
    outpath = opt.outdir

    batch_size = opt.n_samples
    n_rows = opt.n_rows if opt.n_rows > 0 else batch_size
    if not opt.from_file:
        prompt = opt.prompt
        assert prompt is not None
        data = [batch_size * [prompt]]

    else:
        print(f"reading prompts from {opt.from_file}")
        with open(opt.from_file, "r") as f:
            data = f.read().splitlines()
            data = list(chunk(data, batch_size))

    sample_path = os.path.join(outpath, "samples")
    os.makedirs(sample_path, exist_ok=True)
    base_count = len(os.listdir(sample_path))
    grid_count = len(os.listdir(outpath)) - 1

    start_code = None
    if opt.fixed_code:
        start_code = torch.randn([opt.n_samples, opt.C, opt.H // opt.f, opt.W // opt.f], device=device)

    precision_scope = autocast if opt.precision=="autocast" else nullcontext
    with torch.no_grad():
        with precision_scope("cuda"):
            with model.ema_scope():
                tic = time.time()
                all_samples = list()
                for n in trange(opt.n_iter, desc="Sampling"):
                    for prompts in tqdm(data, desc="data"):
                        uc = None
                        if opt.scale != 1.0:
                            uc = model.get_learned_conditioning(batch_size * [""])
                        if isinstance(prompts, tuple):
                            prompts = list(prompts)
                        c = model.get_learned_conditioning(prompts)
                        shape = [opt.C, opt.H // opt.f, opt.W // opt.f]
                        samples_ddim, _ = sampler.sample(S=opt.ddim_steps,
                                                         conditioning=c,
                                                         batch_size=opt.n_samples,
                                                         shape=shape,
                                                         verbose=False,
                                                         unconditional_guidance_scale=opt.scale,
                                                         unconditional_conditioning=uc,
                                                         eta=opt.ddim_eta,
                                                         x_T=start_code)

                        x_samples_ddim = model.decode_first_stage(samples_ddim)
                        x_samples_ddim = torch.clamp((x_samples_ddim + 1.0) / 2.0, min=0.0, max=1.0)

                        if not opt.skip_save:
                            for x_sample in x_samples_ddim:
                                x_sample = 255. * rearrange(x_sample.cpu().numpy(), 'c h w -> h w c')
                                Image.fromarray(x_sample.astype(np.uint8)).save(
                                    os.path.join(sample_path, f"{base_count:05}.png"))
                                base_count += 1

                        if not opt.skip_grid:
                            all_samples.append(x_samples_ddim)

                if not opt.skip_grid:
                    # additionally, save as grid
                    grid = torch.stack(all_samples, 0)
                    grid = rearrange(grid, 'n b c h w -> (n b) c h w')
                    grid = make_grid(grid, nrow=n_rows)

                    # to image
                    grid = 255. * rearrange(grid, 'c h w -> h w c').cpu().numpy()
                    Image.fromarray(grid.astype(np.uint8)).save(os.path.join(outpath, f'grid-{grid_count:04}.png'))
                    grid_count += 1
                    
                    # display
                    if opt.display_inline:
                        clear_output()
                        display(Image.fromarray(grid.astype(np.uint8)))

                toc = time.time()

    print(f"Your samples have been saved to: \n{outpath} \n"
          f" \nEnjoy.")


def run(opt):
    """If the model parameters changed, reload the model, otherwise, just do inference"""

    print(f"Creating image ({opt.H},{opt.W}) from prompt:\n\"{opt.prompt}\"\n")
    
    # FIXME global hack
    global last_config
    global last_ckpt
    global model

    if (opt.config != last_config) or (opt.ckpt != last_ckpt):
        model = load_model(opt)
        # FIXME global hack
        last_config = opt.config
        last_ckpt   = opt.ckpt

    run_inference(opt, model)


# FIXME global hack
last_config = ""
last_ckpt   = ""


####### Widget GUI code #######

def get_widget_extractor(widget_dict):
    # allows accessing after setting, this is to reduce the diff against the argparse code
    class WidgetDict(OrderedDict):
        def __getattr__(self,val):
            return self[val].value
    return WidgetDict(widget_dict)

# Allows long widget descriptions
style  = {'description_width': 'initial'}
# Force widget width to max
layout = widgets.Layout(width='100%')

# args from argparse converted to widgets:
# https://github.com/CompVis/stable-diffusion/blob/main/scripts/txt2img.py#L48-L177
widget_opt = OrderedDict()
widget_opt['prompt'] = widgets.Text(
    layout=layout, style=style,
    description='the prompt to render',
    #value="a painting of a virus monster playing guitar",  # script default
    value="a photograph of an astronaut riding a horse",  # README default
    disabled=False
)
widget_opt['outdir'] = widgets.Text(
    layout=layout, style=style,
    description='dir to write results to',
    value="outputs/txt2img-samples",
    disabled=False
)
widget_opt['skip_grid'] = widgets.Checkbox(
    layout=layout, style=style,
    value=False,
    description='do not save a grid, only individual samples. Helpful when evaluating lots of samples',
    indent=False,
    disabled=False
)
widget_opt['skip_save'] = widgets.Checkbox(
    layout=layout, style=style,
    value=False,
    description='do not save individual samples. For speed measurements.',
    indent=False,
    disabled=False
)
widget_opt['ddim_steps'] = widgets.IntText(
    layout=layout, style=style,
    description='number of ddim sampling steps',
    value=50,
    disabled=False
)
widget_opt['plms'] = widgets.Checkbox(
    layout=layout, style=style,
    value=True,
    description='use plms sampling',
    indent=False,
    disabled=False
)
widget_opt['laion400m'] = widgets.Checkbox(
    layout=layout, style=style,
    value=False,
    description='uses the LAION400M model',
    indent=False,
    disabled=False
)
widget_opt['fixed_code'] = widgets.Checkbox(
    layout=layout, style=style,
    value=False,
    description='if enabled, uses the same starting code across samples',
    indent=False,
    disabled=False
)
widget_opt['ddim_eta'] = widgets.FloatText(
    layout=layout, style=style,
    description='ddim eta (eta=0.0 corresponds to deterministic sampling',
    value=0.0,
    disabled=False
)
widget_opt['n_iter'] = widgets.IntText(
    layout=layout, style=style,
    description='sample this often',
    value=2,
    disabled=False
)
widget_opt['H'] = widgets.IntText(
    layout=layout, style=style,
    description='image height, in pixel space',
    value=512,
    disabled=False
)
widget_opt['W'] = widgets.IntText(
    layout=layout, style=style,
    description='image width, in pixel space',
    value=512,
    disabled=False
)
widget_opt['C'] = widgets.IntText(
    layout=layout, style=style,
    description='latent channels',
    value=4,
    disabled=False
)
widget_opt['f'] = widgets.IntText(
    layout=layout, style=style,
    description='downsampling factor',
    value=8,
    disabled=False
)
widget_opt['n_samples'] = widgets.IntText(
    layout=layout, style=style,
    description='how many samples to produce for each given prompt. A.k.a. batch size',
    value=3,
    disabled=False
)
widget_opt['n_rows'] = widgets.IntText(
    layout=layout, style=style,
    description='rows in the grid (default: n_samples)',
    value=0,
    disabled=False
)
widget_opt['scale'] = widgets.FloatText(
    layout=layout, style=style,
    description='unconditional guidance scale: eps = eps(x, empty) + scale * (eps(x, cond) - eps(x, empty))',
    value=7.5,
    disabled=False
)
widget_opt['from_file'] = widgets.Text(
    layout=layout, style=style,
    description='if specified, load prompts from this file',
    value=None,
    disabled=False
)
widget_opt['config'] = widgets.Text(
    layout=layout, style=style,
    description='path to config which constructs model',
    value="configs/stable-diffusion/v1-inference.yaml",
    disabled=False
)
widget_opt['ckpt'] = widgets.Text(
    layout=layout, style=style,
    description='path to checkpoint of model',
    value="models/ldm/stable-diffusion-v1/model.ckpt",
    disabled=False
)
widget_opt['seed'] = widgets.IntText(
    layout=layout, style=style,
    description='the seed (for reproducible sampling)',
    value=42,
    disabled=False
)
widget_opt['precision'] = widgets.Combobox(
    layout=layout, style=style,
    description='evaluate at this precision',
    value="autocast",
    options=["full", "autocast"],
    disabled=False
)
# Extra option for the notebook
widget_opt['display_inline'] = widgets.Checkbox(
    layout=layout, style=style,
    value=True,
    description='display output images inline (in addition to saving them)',
    indent=False,
    disabled=False
)

# Button that runs the 
# Alternatively, you can just run the following in a new cell:
# run(get_widget_extractor(widget_opt))
run_button = widgets.Button(
    description='CLICK TO DREAM',
    disabled=False,
    button_style='', # 'success', 'info', 'warning', 'danger' or ''
    tooltip='Click to run (settings will update automatically)',
    icon='check'
)
run_button_out = widgets.Output()
def on_run_button_click(b):
    with run_button_out:
        clear_output()
        run(get_widget_extractor(widget_opt))
run_button.on_click(on_run_button_click)

# Package into box and render
#primary_options = ['prompt', 'outdir']  # options to put up top
#secondary_options = [k for k in widget_opt.keys() if k not in primary_options]  # rest, ordered by insertion

load_options = ['config', 'ckpt']
inference_options = [k for k in widget_opt.keys() if k not in load_options]  # rest, ordered by insertion
assert all([k in inference_options + load_options for k in widget_opt.keys()])  # make sure we didn't miss any options

# Package into box for rendering
gui = widgets.VBox(
    [widget_opt[k] for k in inference_options] + [widget_opt[k] for k in load_options]  # + [run_button, run_button_out]
)

## Interactive loop

Change options using the GUI, then run the next cell - **no need to re-run/display the GUI cell (the GUI will automatically update the variables)**

**Just edit the options, then run the next cell (`Run to start dreaming`) to run the model.**

In [3]:
#@title Display options GUI { vertical-output: true, display-mode: "form" }

os.chdir('/workspace/stable-diffusion')
display(gui)

In [4]:
#@title Run to start dreaming { vertical-output: true, display-mode: "form" }
run(get_widget_extractor(widget_opt))

Creating image (512,512) from prompt:
"a photograph of an astronaut riding a horse"

Loading model from models/ldm/stable-diffusion-v1/model.ckpt
Global Step: 440000
LatentDiffusion: Running in eps-prediction mode
DiffusionWrapper has 859.52 M params.
making attention of type 'vanilla' with 512 in_channels
Working with z of shape (1, 4, 32, 32) = 4096 dimensions.
making attention of type 'vanilla' with 512 in_channels


Downloading: "https://github.com/DagnyT/hardnet/raw/master/pretrained/train_liberty_with_aug/checkpoint_liberty_with_aug.pth" to /root/.cache/torch/hub/checkpoints/checkpoint_liberty_with_aug.pth


  0%|          | 0.00/5.10M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/939k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/512k [00:00<?, ?B/s]

...

Data shape for PLMS sampling is (3, 4, 64, 64)
Running PLMS Sampling with 100 timesteps




PLMS Sampler:   0%|          | 0/100 [00:00<?, ?it/s]

PLMS Sampler:   1%|          | 1/100 [00:02<04:03,  2.46s/it]

PLMS Sampler:   2%|▏         | 2/100 [00:02<01:52,  1.15s/it]

PLMS Sampler:   3%|▎         | 3/100 [00:02<01:10,  1.37it/s]

PLMS Sampler:   4%|▍         | 4/100 [00:03<00:51,  1.88it/s]

PLMS Sampler:   5%|▌         | 5/100 [00:03<00:43,  2.17it/s]

PLMS Sampler:   6%|▌         | 6/100 [00:03<00:36,  2.61it/s]

PLMS Sampler:   7%|▋         | 7/100 [00:03<00:31,  3.00it/s]

PLMS Sampler:   8%|▊         | 8/100 [00:04<00:27,  3.32it/s]

PLMS Sampler:   9%|▉         | 9/100 [00:04<00:25,  3.58it/s]

PLMS Sampler:  10%|█         | 10/100 [00:04<00:23,  3.77it/s]

PLMS Sampler:  11%|█         | 11/100 [00:04<00:22,  3.93it/s]

PLMS Sampler:  12%|█▏        | 12/100 [00:05<00:21,  4.03it/s]

PLMS Sampler:  13%|█▎        | 13/100 [00:05<00:21,  4.12it/s]

PLMS Sampler:  14%|█▍        | 14/100 [00:05<00:20,  4.18it/s]

PLMS Sampler:  15%|█▌        | 15/100 [00:05<00:20,  4.2

Data shape for PLMS sampling is (3, 4, 64, 64)
Running PLMS Sampling with 100 timesteps




PLMS Sampler:   0%|          | 0/100 [00:00<?, ?it/s]

PLMS Sampler:   1%|          | 1/100 [00:00<00:45,  2.19it/s]

PLMS Sampler:   2%|▏         | 2/100 [00:00<00:31,  3.08it/s]

PLMS Sampler:   3%|▎         | 3/100 [00:00<00:27,  3.54it/s]

PLMS Sampler:   4%|▍         | 4/100 [00:01<00:25,  3.81it/s]

PLMS Sampler:   5%|▌         | 5/100 [00:01<00:23,  3.97it/s]

PLMS Sampler:   6%|▌         | 6/100 [00:01<00:23,  4.08it/s]

PLMS Sampler:   7%|▋         | 7/100 [00:01<00:22,  4.15it/s]

PLMS Sampler:   8%|▊         | 8/100 [00:02<00:21,  4.20it/s]

PLMS Sampler:   9%|▉         | 9/100 [00:02<00:21,  4.23it/s]

PLMS Sampler:  10%|█         | 10/100 [00:02<00:21,  4.25it/s]

PLMS Sampler:  11%|█         | 11/100 [00:02<00:20,  4.25it/s]

PLMS Sampler:  12%|█▏        | 12/100 [00:03<00:20,  4.26it/s]

PLMS Sampler:  13%|█▎        | 13/100 [00:03<00:20,  4.27it/s]

PLMS Sampler:  14%|█▍        | 14/100 [00:03<00:20,  4.28it/s]

PLMS Sampler:  15%|█▌        | 15/100 [00:03<00:19,  4.2

Your samples have been saved to: 
outputs/txt2img-samples 
 
Enjoy.
